In [1]:
import numpy as np
from tinygrad.helpers import Timing
from tinygrad import Tensor

In [2]:
t1 = Tensor([1, 2, 3, 4, 5])
na = np.array([1, 2, 3, 4, 5])
t2 = Tensor(na)

In [3]:
full = Tensor.full(shape=(2, 3), fill_value=5) # create a tensor of shape (2, 3) filled with 5
zeros = Tensor.zeros(2, 3) # create a tensor of shape (2, 3) filled with 0
ones = Tensor.ones(2, 3) # create a tensor of shape (2, 3) filled with 1

full_like = Tensor.full_like(full, fill_value=2) # create a tensor of the same shape as `full` filled with 2
zeros_like = Tensor.zeros_like(full) # create a tensor of the same shape as `full` filled with 0
ones_like = Tensor.ones_like(full) # create a tensor of the same shape as `full` filled with 1

eye = Tensor.eye(3) # create a 3x3 identity matrix
arange = Tensor.arange(start=0, stop=10, step=1) # create a tensor of shape (10,) filled with values from 0 to 9

rand = Tensor.rand(2, 3) # create a tensor of shape (2, 3) filled with random values from a uniform distribution
randn = Tensor.randn(2, 3) # create a tensor of shape (2, 3) filled with random values from a standard normal distribution
uniform = Tensor.uniform(2, 3, low=0, high=10) # create a tensor of shape (2, 3) filled with random values from a uniform distribution between 0 and 10

In [4]:
from tinygrad import dtypes

t3 = Tensor([1, 2, 3, 4, 5], dtype=dtypes.int32)

In [5]:
t4 = Tensor([1, 2, 3, 4, 5])
t5 = (t4 + 1) * 2
t6 = (t5 * t4).relu().log_softmax()

In [6]:
class Linear:
  def __init__(self, in_features, out_features, bias=True, initialization: str='kaiming_uniform'):
    self.weight = getattr(Tensor, initialization)(out_features, in_features)
    self.bias = Tensor.zeros(out_features) if bias else None

  def __call__(self, x:Tensor):
    return x.linear(self.weight.transpose(), self.bias)

In [42]:
t.tolist()

[[1, 2], [3, 4]]

In [7]:
t = Tensor([[1, 2], [3, 4]]) # sample x features
weight = Tensor([[1, 2], [3, 4], [5, 5]]).transpose() # 
bias = Tensor([1, 2, 6])

In [8]:
# a row represents a sample input, 
# each colunm is a neuron value
print(t.linear(weight, bias).numpy()) # t * w + b

[[ 6 13 21]
 [12 27 41]]


In [9]:
class TinyNet:
  def __init__(self):
    self.l1 = Linear(784, 128, bias=False)
    self.l2 = Linear(128, 10, bias=False)

  def __call__(self, x:Tensor):
    x = self.l1(x)
    x = x.leakyrelu()
    x = self.l2(x)
    return x

net = TinyNet()

In [10]:
def sparse_categorical_crossentropy(self, Y:Tensor, ignore_index=-1) -> Tensor:
    loss_mask = Y != ignore_index
    y_counter = Tensor.arange(self.shape[-1], dtype=dtypes.int32, requires_grad=False, device=self.device).unsqueeze(0).expand(Y.numel(), self.shape[-1])
    y = ((y_counter == Y.flatten().reshape(-1, 1)).where(-1.0, 0) * loss_mask.reshape(-1, 1)).reshape(*Y.shape, self.shape[-1])
    return self.log_softmax().mul(y).sum() / loss_mask.sum()

In [11]:
from tinygrad.nn.optim import SGD

opt = SGD([net.l1.weight, net.l2.weight], lr=3e-4)
# instead of passing params explicitly you can just do tinigrad.nn.state.get_parameters(net)

In [12]:
from extra.datasets import fetch_mnist

c:\Users\acl\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
X_train, Y_train, X_test, Y_test = fetch_mnist()
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz: 1
https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz: 1
https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz: 10
https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz: 10


((60000, 784), (60000,), (10000, 784), (10000,))

In [14]:
with Tensor.train():
  for step in range(1000):
    # random sample a batch
    samp = np.random.randint(0, X_train.shape[0], size=(64))
    batch = Tensor(X_train[samp], requires_grad=False)
    # get the corresponding labels
    labels = Tensor(Y_train[samp])

    # forward pass
    out = net(batch)

    # compute loss
    loss = sparse_categorical_crossentropy(out, labels)

    # zero gradients
    opt.zero_grad()

    # backward pass
    loss.backward()

    # update parameters
    opt.step()

    # calculate accuracy
    pred = out.argmax(axis=-1)
    acc = (pred == labels).mean()

    if step % 100 == 0:
      print(f"Step {step+1} | Loss: {loss.numpy()} | Accuracy: {acc.numpy()}")

Step 1 | Loss: 165.96644592285156 | Accuracy: 0.09375
Step 101 | Loss: 6.453737258911133 | Accuracy: 0.796875
Step 201 | Loss: 13.302399635314941 | Accuracy: 0.734375
Step 301 | Loss: 6.442187309265137 | Accuracy: 0.8125
Step 401 | Loss: 4.340219020843506 | Accuracy: 0.828125
Step 501 | Loss: 4.145925045013428 | Accuracy: 0.8125
Step 601 | Loss: 2.3986942768096924 | Accuracy: 0.90625
Step 701 | Loss: 2.5127370357513428 | Accuracy: 0.828125
Step 801 | Loss: 1.9548487663269043 | Accuracy: 0.90625
Step 901 | Loss: 0.7563364505767822 | Accuracy: 0.921875


In [16]:
with Timing("Time: "):
  avg_acc = 0
  for step in range(1000):
    # random sample a batch
    samp = np.random.randint(0, X_test.shape[0], size=(64))
    batch = Tensor(X_test[samp], requires_grad=False)
    # get the corresponding labels
    labels = Y_test[samp]

    # forward pass
    out = net(batch)

    # calculate accuracy
    pred = out.argmax(axis=-1).numpy()
    avg_acc += (pred == labels).mean()
  print(f"Test Accuracy: {(avg_acc / 1000):.2%}")

Test Accuracy: 88.89%
Time: 6298.67 ms


Now look at `examples/`, `test/` and `extra/`